In [1]:
!rm -r /content/train
!rm -r /content/test

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget --no-check-certificate \
        https://www.dropbox.com/s/8u2k79tuqmwrwi8/train.zip \
       -O /tmp/train.zip

--2021-02-13 12:49:49--  https://www.dropbox.com/s/8u2k79tuqmwrwi8/train.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8u2k79tuqmwrwi8/train.zip [following]
--2021-02-13 12:49:49--  https://www.dropbox.com/s/raw/8u2k79tuqmwrwi8/train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbeae956d4f602b1f3aeac1451f.dl.dropboxusercontent.com/cd/0/inline/BI1zcs0WM8TRjleL1p0nWh7i5qqVF-FlZ6xE6OzYObotQYQZUXh2OTpfRsawRcZACim2lk4JBJT_LRhAUXwEVr3BW_iAuEeWv-KZQAgSRvEp5mW2Mhwf1vdE_fiuTQ7YvMg/file# [following]
--2021-02-13 12:49:50--  https://ucbeae956d4f602b1f3aeac1451f.dl.dropboxusercontent.com/cd/0/inline/BI1zcs0WM8TRjleL1p0nWh7i5qqVF-FlZ6xE6OzYObotQYQZUXh2OTpfRsawRcZACim2lk4JBJT_LRhAUXwEVr3BW_iAuEeWv-KZQAgSRvEp5mW2Mhwf1vdE

In [3]:
!wget --no-check-certificate \
        https://www.dropbox.com/s/pnipjr7brjz1pm5/test.zip \
       -O /tmp/test.zip

--2021-02-13 12:49:56--  https://www.dropbox.com/s/pnipjr7brjz1pm5/test.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pnipjr7brjz1pm5/test.zip [following]
--2021-02-13 12:49:56--  https://www.dropbox.com/s/raw/pnipjr7brjz1pm5/test.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb2bcd24eb0a33f9cc91a7b488c.dl.dropboxusercontent.com/cd/0/inline/BI0nkVymd-sOUsuOBxkyLB8UjqMIIf3VgqzGJJI4jK6C1_IaANn03kkM7y5Gxg2nPBoZFxqSoGxFyqQxKFFvzgnM54uBb35bxAM1VBjIzmM5zG_oPWDkfY885T1EhMqWWzs/file# [following]
--2021-02-13 12:49:56--  https://ucb2bcd24eb0a33f9cc91a7b488c.dl.dropboxusercontent.com/cd/0/inline/BI0nkVymd-sOUsuOBxkyLB8UjqMIIf3VgqzGJJI4jK6C1_IaANn03kkM7y5Gxg2nPBoZFxqSoGxFyqQxKFFvzgnM54uBb35bxAM1VBjIzmM5zG_oPWDkfY885T1

In [4]:
import zipfile

with zipfile.ZipFile("//tmp/train.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")
zip_ref.close()

with zipfile.ZipFile("//tmp/test.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")
zip_ref.close()

In [5]:
TRAIN_DIR = '/content/train'
TEST_DIR = '/content/test'

In [19]:
BATCH_SIZE = 32

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=.2)

train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True, subset='training'
)

val_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True, subset='validation'
)

Found 13210 images belonging to 32 classes.
Found 3286 images belonging to 32 classes.


In [22]:
test_datagen = ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_directory(
    directory=TEST_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True
)

Found 4142 images belonging to 32 classes.


In [23]:
MAP_CHARACTERS = {v:k for k, v in train_generator.class_indices.items()}

In [24]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [25]:
tf.keras.backend.clear_session()

In [26]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(.1),
    tf.keras.layers.Dense(len(MAP_CHARACTERS), activation='softmax')
])

In [27]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
train_generator.samples

13210

In [30]:
model.fit(train_generator,
          validation_data=val_generator,
          steps_per_epoch=train_generator.samples / BATCH_SIZE,
          epochs=15)

Epoch 1/15
412/412 [==============================] - 201s 486ms/step - loss: 3.4389 - accuracy: 0.2197 - val_loss: 2.0613 - val_accuracy: 0.4306
Epoch 2/15
412/412 [==============================] - 199s 481ms/step - loss: 1.8379 - accuracy: 0.4837 - val_loss: 1.8510 - val_accuracy: 0.4933
Epoch 3/15
412/412 [==============================] - 199s 482ms/step - loss: 1.5520 - accuracy: 0.5595 - val_loss: 1.6773 - val_accuracy: 0.5514
Epoch 4/15
412/412 [==============================] - 199s 482ms/step - loss: 1.2953 - accuracy: 0.6358 - val_loss: 1.6433 - val_accuracy: 0.5682
Epoch 5/15
412/412 [==============================] - 200s 484ms/step - loss: 1.1294 - accuracy: 0.6760 - val_loss: 1.5591 - val_accuracy: 0.5749
Epoch 6/15
412/412 [==============================] - 200s 484ms/step - loss: 0.9837 - accuracy: 0.7156 - val_loss: 1.5159 - val_accuracy: 0.5968
Epoch 7/15
412/412 [==============================] - 201s 486ms/step - loss: 0.8813 - accuracy: 0.7470 - val_loss: 1.5109 -

In [35]:
import os
save_path = os.path.join("//content/drive/MyDrive/simpsonsnet/1/")
tf.saved_model.save(model, save_path)

INFO:tensorflow:Assets written to: //content/drive/MyDrive/simpsonsnet/1/assets
